In [4]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold
iris = datasets.load_iris() #데이터를 로드
features = iris.data # 특성과 타깃을 만듭니다
target = iris.target
thresholder = VarianceThreshold(threshold=.6) # 기준값을 만듭니다.
features_high_variance = thresholder.fit_transform(features) # 기준값보다 높은 특성을 선택합니다.
features_high_variance[0:3] # 선택한 특성을 확인
thresholder.variances_ # 분산 확인

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # 특성 행렬을 표준화합니다.
features_std = scaler.fit_transform(features)
selector = VarianceThreshold() # 각 특성의 분산을 계산합니다.
selector.fit(features_std).variances_ # 특성을 표준화하면 평균이 0으로 모든 분산이 1로 맞추어 짐

array([1., 1., 1., 1.])

In [5]:
features_high_variance[0:3] 

array([[5.1, 1.4],
       [4.9, 1.4],
       [4.7, 1.3]])

In [12]:
from sklearn.feature_selection import VarianceThreshold
import numpy as np
import pandas as pd

features = [[0, 1, 0], # 예제 특성 행렬
            [0, 1, 1], # 특성 0: 80%가 클래스 0
            [0, 1, 0], # 특성 1: 80%가 클래스 1
            [0, 1, 1], # 특성 2: 60%가 클래스 0, 40%는 클래스 1
            [1, 0, 0]]
# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)
thresholder.variances_

np.var(features, axis=0) 

array([0.16, 0.16, 0.24])

In [14]:
# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1], [2, 2, 0], [3, 3, 1], [4, 4, 0], [5, 5, 1],
[6, 6, 0], [7, 7, 1], [8, 7, 0], [9, 7, 1]])
dataframe = pd.DataFrame(features) # 특성 행렬을 DataFrame으로 변환
corr_matrix = dataframe.corr().abs() # 상관관계 행렬을 만듭니다.
# 상관관계 행렬의 상삼각(upper triangle) 행렬을 선택합니다.
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

[1]

In [15]:
upper

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


In [20]:
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [21]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

iris = load_iris() # 데이터 로드
features = iris.data
target = iris.target
features = features.astype(int) # 범주형 데이터를 정수형으로 변환
chi2_selector = SelectKBest(chi2, k=2) # 카이제곱 통계값이 가장 큰 특성 두 개를 선택
features_kbest = chi2_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [22]:
# F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])
# 특정 특성 개수를 선택하는 대신 Selectpercentile를 사용하여 특성의 상위 n 퍼센트를 선택할 수 있습니다.

from sklearn.feature_selection import SelectPercentile

# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 선택
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2
원본 특성 개수: 4
줄어든 특성 개수: 3


In [24]:
target
#특성 행렬의 차원을 (3, 50, 4)로 바꾸어 클래스별 합을 구합니다.
observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed
#특성 타깃과 전혀 관계없다면 기대 빈도는 전체 합을 클래스 개수 3으로 나눈 값이 됩니다.
expected = features.sum(axis=0) / 3
expected
#카이제곱 공식에 위헤서 구한 observed와 expected를 대입합니다.
np.sum((observed - expected)**2 / expected, axis=0)
#카이제곱 값이 큰 세 번째, 네 번째 특성이 선택됩니다. chi2_selector객체의 scores_속성에 저장
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [26]:
print(observed, '\n',expected)

[[230 152  50   0]
 [274 116 191  50]
 [304 129 255  79]] 
 [269.33333333 132.33333333 165.33333333  43.        ]


In [32]:
#ANOVA 직접 계산
##전체 평균과 클래스 평균을 계산
total_mean = np.mean(features, axis=0)
total_mean
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean
#ss_total 계산
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between
ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total
#ss_beteen과 ss_tatal을 F-값 공식에 대입
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
print(f)
print(fvalue_selector.scores_) #F-값 scores_속성에서 확인

[  81.19776715   33.71497585 1160.00645624  385.48275862]
[  81.19776715   33.71497585 1160.00645624  385.48275862]


In [50]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model
import warnings

warnings.simplefilter('ignore')

# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 10000,
n_features = 100,
n_informative = 2,
random_state = 1)
# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()
# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

array([[ 0.00850799,  0.7031277 ,  0.70243828, -0.34606121],
       [-1.07500204,  2.56148527, -0.07152579, -1.8392567 ],
       [ 1.37940721, -1.77039484, -0.00791373, -0.90016708],
       ...,
       [-0.80331656, -1.60648007, -1.3494921 , -1.28329706],
       [ 0.39508844, -1.34564911, -0.55243048,  0.85012142],
       [-0.55383035,  0.82880112, -0.5175125 ,  0.27741159]])

In [53]:
rfecv.n_features_ # 최선의 특성 개수
rfecv.support_ # 선택된 특성이 표시된 불리언 마스크
rfecv.ranking_ # 특성의 순위: 최고(1)에서 최악(96)까지

from sklearn.feature_selection import RFE

rfe = RFE(estimator=ols, n_features_to_select=4)
rfe.fit(features, target)
rfe.transform(features)
# rfe객체가 선택한 특성이 rfecv 객체가 선택한 특성과 동일한지 확인하기 위해 불리언 마스크를 비교
np.all(rfe.support_ == rfecv.support_)

True

In [51]:
rfecv.n_features_

4

In [56]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

digits = datasets.load_digits() # 데이터셋 로드
features = digits.data # 특성 행렬을 만듭니다.
target = digits.target # 타깃 벡터를 만듭니다.
standardizer = StandardScaler() # 표준화 객체를 만듭니다.
logit = LogisticRegression() # # 로지스틱 회귀 객체를 만듭니다

# 표준화한 다음 로지스틱 회귀를 실행하는 파이프라인을 만듭니다.
pipeline = make_pipeline(standardizer, logit)

kf = KFold(n_splits=10, shuffle=True, random_state=1) # k-폴드 교차검증을 만듭니다. (10 개의 폴드를 만듬)

# k-폴드 교차검증을 수행합니다.
cv_results = cross_val_score(pipeline, features, target, cv=kf, # 교차 검증 기법
                            scoring="accuracy", # 평가 지표
                            n_jobs=-1) # 모든 CPU 코어 사용

cv_results.mean() # 평균을 계산
cv_results ## 10개 폴드의 점수를 모두 확인(평가 점수는 cv_results에 저장)

array([0.97222222, 0.97777778, 0.95555556, 0.95      , 0.95555556,
       0.98333333, 0.97777778, 0.96648045, 0.96089385, 0.94972067])

In [58]:
from sklearn.model_selection import train_test_split
# 훈련 세트와 테스트 세트를 만듭니다.

features_train, features_test, target_train, target_test = train_test_split( features, target, test_size=0.1, random_state=1)
standardizer.fit(features_train) # 훈련 세트로 standardizer의 fit 메서드를 호출

# 훈련 세트와 테스트 세트에 모두 적용합니다.
features_train_std = standardizer.transform(features_train)
features_test_std = standardizer.transform(features_test)
pipeline = make_pipeline(standardizer, logit) # 파이프라인을 만듭니다.

# k-폴드 교차 검증 수행
cv_results = cross_val_score(pipeline, # 파이프라인
                            features, # 특성 행렬
                            target, # 타깃 벡터
                            cv=kf, # 교차검증
                            scoring="accuracy", # 평가 지표
                            n_jobs=-1) # 모든 CPU 코어 사용b

cv_results

array([0.97222222, 0.97777778, 0.95555556, 0.95      , 0.95555556,
       0.98333333, 0.97777778, 0.96648045, 0.96089385, 0.94972067])

In [63]:
#실습 : 로지스틱 회귀에서 C와 규제 페널티 값의 각 조합에 대해 모델을 훈련하고 k-폴드 교차검증으로 평가합니다.
#C의 값이 10개이고 규제 페널티는 두 개, 폴드 수는 5입니다. 총 10X2X5=100개의 모델 후보 중에서 가장 좋은 것을
#선택합니다.
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV

iris = datasets.load_iris() # 데이터 로드
features = iris.data
target = iris.target
logistic = linear_model.LogisticRegression() # 로지스틱 회귀 모델 객체 생성
penalty = ['l1', 'l2'] # 페널티(penalty) 하이퍼파라미터 값의 후보를 만듭니다.
C = np.logspace(0, 4, 10) # 규제 하이퍼파라미터 값의 후보 범위를 만듭니다.
hyperparameters = dict(C=C, penalty=penalty)
hyperparameters

{'C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
        5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
        3.59381366e+03, 1.00000000e+04]), 'penalty': ['l1', 'l2']}

In [72]:
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, verbose=1) # 그리드 서치 객체 생성
best_model = gridsearch.fit(features, target) # 그리드 서치 수행 .
np.logspace(0, 4, 10)
# 최선의 하이퍼파라미터를 확인합니다.
print('가장 좋은 페널티:', best_model.best_estimator_.get_params()['penalty'])
print('가장 좋은 C 값:', best_model.best_estimator_.get_params()['C'])
best_model.predict(features) # 타깃 벡터 예측

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


가장 좋은 페널티: l1
가장 좋은 C 값: 7.742636826811269


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [67]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
np.random.seed(0) # 랜덤 시드 설정
iris = datasets.load_iris() # 데이터 로드
features = iris.data
target = iris.target
pipe = Pipeline([("classifier", RandomForestClassifier())]) # 파이프라인을 만듭니다.
# 후보 학습 알고리즘과 하이퍼파라미터로 딕셔너리를 만듭니다.
search_space = [ { "classifier": [LogisticRegression()],
"classifier__penalty": ['l1', 'l2'],
"classifier__C": np.logspace(0, 4, 10)},
{ "classifier": [RandomForestClassifier()],
"classifier__n_estimators": [10, 100, 1000],
"classifier__max_features": [1, 2, 3]}]

In [74]:
gridsearch = GridSearchCV(pipe, search_space, cv=5, verbose=1) # 그리드 서치 객체 생성
best_model = gridsearch.fit(features, target) # 그리드 서치를 수행
best_model.best_estimator_.get_params()["classifier"] # 최선의 모델을 확인
best_model.predict(features) # 타깃 벡터를 예측

Fitting 5 folds for each of 29 candidates, totalling 145 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 145 out of 145 | elapsed:   23.9s finished


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [75]:
best_model.best_estimator_.get_params()["classifier"]

LogisticRegression(C=7.742636826811269, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

np.random.seed(0) # 랜덤 시드 설정
iris = datasets.load_iris() # 데이터 로드
features = iris.data
target = iris.target

# StandardScaler와 PCA를 포함한 전처리 객체를 만듭니다.
preprocess = FeatureUnion([("std", StandardScaler()), ("pca", PCA())])

# 파이프라인을 만듭니다.
pipe = Pipeline([("preprocess", preprocess), ("classifier", LogisticRegression())])

In [87]:
# 후보 값을 정의합니다.
#최선의 모델을 만드는 주성분이 하나인지 두개, 세 개인지를 탐색하도록 지시합니다.
search_space = [{"preprocess__pca__n_components": [1, 2, 3],
                "classifier__penalty": ["l1", "l2"],
                "classifier__C": np.logspace(0, 4, 10)}]

clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1) # 그리드 서치 객체 생성

best_model = clf.fit(features, target) # 그리드 서치 수행

best_model.best_estimator_.get_params()['preprocess__pca__n_components'] # 최선의 주성분 개수를 확인
clf.best_score_ #GridSearchCV가 수행한 교차검증에서 최상의 점수가 저장되는 속성

clf.best_estimator_.named_steps["preprocess"].transform(features[0:1])
# memory 매개변수에 전처리 데이터를 임시 저장할 디렉토리 이름을 전달하면 하이퍼파라미터 탐색 과정에서
# 중복으로 전처리 과정을 수행하지 않습니다
pipe = Pipeline([("std", StandardScaler()),
                ("pca", PCA()),
                ("classifier", LogisticRegression())],
                memory='cache')

In [94]:
# 후보 값을 정의합니다.
search_space = [{"pca__n_components": [1, 2, 3],
                "classifier__penalty": ["l1", "l2"],
                "classifier__C": np.logspace(0, 4, 10)}]

clf = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1) # 그리드 서치 객체생성
best_model = clf.fit(features, target) # 그리드 서치 수행

clf.best_score_

clf.best_estimator_.get_params()['pca__n_components'] # 최선의 주성분 개수를 확인

clf.best_estimator_.named_steps["pca"].transform(features[0:1])

array([[ 2.64026976,  5.2040413 , -2.48862071]])

In [100]:
%%time
import numpy as np
from sklearn import linear_model, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris() # 데이터를 로드
features = iris.data
target = iris.target
logistic = linear_model.LogisticRegression() # 로지스틱 회귀 모델 객체 생성
penalty = ["l1", "l2"] # 규제 페널티의 후보를 만듭니다.
C = np.logspace(0, 4, 1000) # C 값의 후보 범위를 만듭니다.
hyperparameters = dict(C=C, penalty=penalty) # 하이퍼파라미터 옵션을 만듭니다.
gridsearch = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=-1, verbose=1)
best_model = gridsearch.fit(features, target)
# 하나의 코어만 사용하는 그리드 서치 객체를 만듭니다.


Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 312 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 5112 tasks      | elapsed:   15.5s


CPU times: user 7.2 s, sys: 40.4 ms, total: 7.24 s
Wall time: 34.7 s


[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:   34.6s finished


In [101]:
%%time
clf = GridSearchCV(logistic, hyperparameters, cv=5, n_jobs=1, verbose=1)
best_model = clf.fit(features, target) # 그리드 서치를 수행

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


CPU times: user 1min 17s, sys: 38.1 ms, total: 1min 17s
Wall time: 1min 17s


[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:  1.3min finished
